In [68]:
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import mapclassify as mc
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.colors import ListedColormap

In [69]:
modelFolderPath = "Accessibility_Dissolve_Animation"

In [70]:
chicagoNameList = glob.glob(os.path.join(modelFolderPath,"Chicago_ACC.*.shp"))
illinoisNameList = glob.glob(os.path.join(modelFolderPath,"Illinois_ACC.*.shp"))
chicagoList = []
illinoisList = []

for i in range(0, len(chicagoNameList)):
    chicagoList.append(glob.glob(os.path.join(modelFolderPath,"Chicago_ACC.*.shp"))[i][45:55])
for i in range(0, len(illinoisNameList)):
    illinoisList.append(glob.glob(os.path.join(modelFolderPath,"Illinois_ACC.*.shp"))[i][46:56])
    
print(chicagoList)
print(illinoisList)

['05-26-2020', '05-27-2020', '05-28-2020', '05-29-2020', '05-30-2020', '05-31-2020', '06-01-2020', '06-02-2020', '06-03-2020']
['05-26-2020', '05-27-2020', '05-28-2020', '05-29-2020', '05-30-2020', '05-31-2020', '06-01-2020', '06-02-2020', '06-03-2020']


In [71]:
def selectFirstDate(nameList):
    dmin = datetime.strptime(nameList[0], "%m-%d-%Y").date()
    index = 0
    for i in range(0, len(nameList)):
        d = datetime.strptime(nameList[i], "%m-%d-%Y").date()
        if (d<=dmin):
            index = i
            dmin = d
    return index

def selectLastDate(nameList):
    dmax = datetime.strptime(nameList[0], "%m-%d-%Y").date()
    index = 0
    for i in range(0, len(nameList)):
        d = datetime.strptime(nameList[i], "%m-%d-%Y").date()
        if (d>=dmax):
            index = i
            dmax = d
    return index

In [72]:
firstIndex = selectFirstDate(chicagoList)
lastIndex = selectLastDate(chicagoList)
df = gpd.read_file(glob.glob(os.path.join(modelFolderPath,"Chicago_ACC."+chicagoList[lastIndex]+"*.shp"))[0])
nbArray = mc.NaturalBreaks(df['normal_val'], k=6).bins.astype(float)

def classifyChicago(value):
    if value >= 0 and value <= nbArray[0]:
        return 0
    if value >= nbArray[0] and value <= nbArray[1]:
        return 1
    elif value > nbArray[1] and value <= nbArray[2]:
        return 2
    elif value > nbArray[2] and value <= nbArray[3]:
        return 3
    elif value > nbArray[3] and value <= nbArray[4]:
        return 4
    else:
        return 5

def classRangeChicago(value):
    if value == 0:
        return "0~"+str(round(nbArray[0],2))
    if value == 1:
        return str(round(nbArray[0],2))+"~"+str(round(nbArray[1],2))
    elif value == 2:
        return str(round(nbArray[1],2))+"~"+str(round(nbArray[2],2))
    elif value == 3:
        return str(round(nbArray[2],2))+"~"+str(round(nbArray[3],2))
    elif value == 4:
        return str(round(nbArray[3],2))+"~"+str(round(nbArray[4],2))
    else:
        return str(round(nbArray[4],2))+"~"+str(round(nbArray[5],2))

chicagoACC_dissolve_list = []    

for i in range(0, len(chicagoList)):
    #In case we have multiple files for one day
    if (i != 0):
        if (chicagoList[i] == chicagoList[i-1]):
            continue
    chicagoACC = gpd.read_file(glob.glob(os.path.join(modelFolderPath,"Chicago_ACC."+chicagoList[i]+"*.shp"))[0])
    #chicagoACC = chicagoACC.rename(columns={"normal_cov": "normal_val"})
    
    chicagoACC['category'] = chicagoACC.apply(lambda x: classifyChicago(x.normal_val), axis = 1)
    chicagoACC_dissolve = chicagoACC.dissolve(by='category', aggfunc = 'mean')
    chicagoACC_dissolve = chicagoACC_dissolve.drop(columns=['id'])
    chicagoACC_dissolve["category"] = range(0, len(chicagoACC_dissolve))
    chicagoACC_dissolve['date'] = datetime.strptime(chicagoList[i], "%m-%d-%Y").date().strftime("%Y-%m-%d")
    chicagoACC_dissolve['range'] = chicagoACC_dissolve.apply(lambda x: classRangeChicago(x.category), axis = 1)
    
    chicagoACC_dissolve = chicagoACC_dissolve.to_crs({'init': 'epsg:4326'})
    #chicagoACC_dissolve.to_file("Chicago_ACC."+chicagoList[i]+"_dissolve.geojson", driver="GeoJSON")
    
    chicagoACC_dissolve_list.append(chicagoACC_dissolve)
    
chicagoACC_concat = pd.concat(chicagoACC_dissolve_list, axis=0)
chicagoACC_concat = chicagoACC_concat.drop(columns=['category'])
chicagoACC_concat = chicagoACC_concat.reset_index()

#delta = timedelta(days=len(chicagoList))
firstDate = datetime.strptime(chicagoList[firstIndex], "%m-%d-%Y").date()
lastDate = datetime.strptime(chicagoList[lastIndex], "%m-%d-%Y").date()
chicagoACC_concat["dt_start"] = firstDate.strftime("%Y-%m-%d") #(lastDate-delta).strftime("%Y-%m-%d")
chicagoACC_concat["dt_end"] = lastDate.strftime("%Y-%m-%d")
chicagoACC_concat["dt_unit"] = "day"
chicagoACC_concat["cases_ts"] = "0,0,0,0,0,0,0"#str(np.zeros(len(chicagoList),int)).replace(" ",",")
chicagoACC_concat.to_file("Chicago_ACC_dissolve.geojson", driver="GeoJSON")

print("done")

done


In [73]:
chicagoACC_concat

,category,geometry,normal_val,date,range,dt_start,dt_end,dt_unit,cases_ts
0,0,"MULTIPOLYGON (((-87.66365 41.65347, -87.66194 ...",0.006553,2020-05-26,0~0.13,2020-05-26,2020-06-03,day,"0,0,0,0,0,0,0"
1,1,"MULTIPOLYGON (((-87.66897 41.66470, -87.66726 ...",0.266113,2020-05-26,0.13~0.36,2020-05-26,2020-06-03,day,"0,0,0,0,0,0,0"
2,2,"MULTIPOLYGON (((-87.60877 41.89921, -87.60705 ...",0.456791,2020-05-26,0.36~0.53,2020-05-26,2020-06-03,day,"0,0,0,0,0,0,0"
3,3,"MULTIPOLYGON (((-87.61934 41.91267, -87.61762 ...",0.604973,2020-05-26,0.53~0.67,2020-05-26,2020-06-03,day,"0,0,0,0,0,0,0"
4,4,"MULTIPOLYGON (((-87.61758 41.91043, -87.61410 ...",0.742211,2020-05-26,0.67~0.82,2020-05-26,2020-06-03,day,"0,0,0,0,0,0,0"
5,5,"POLYGON ((-87.66398 41.85387, -87.66570 41.851...",0.903208,2020-05-26,0.82~1.0,2020-05-26,2020-06-03,day,"0,0,0,0,0,0,0"
6,0,"MULTIPOLYGON (((-87.66365 41.65347, -87.66194 ...",0.006553,2020-05-27,0~0.13,2020-05-26,2020-06-03,day,"0,0,0,0,0,0,0"
7,1,"MULTIPOLYGON (((-87.66897 41.66470, -87.66726 ...",0.265899,2020-05-27,0.13~0.36,2020-05-26,2020-06-03,day,"0,0,0,0,0,0,0"
8,2,"MULTIPOLYGON (((-87.60877 41.89921, -87.60705 ...",0.456565,2020-05-27,0.36~0.53,2020-05-26,2020-06-03,day,"0,0,0,0,0,0,0"
9,3,"MULTIPOLYGON (((-87.61934 41.91267, -87.61762 ...",0.604893,2020-05-27,0.53~0.67,2020-05-26,2020-06-03,day,"0,0,0,0,0,0,0"


In [74]:
firstIndex = selectFirstDate(illinoisList)
lastIndex = selectLastDate(illinoisList)
df = gpd.read_file(glob.glob(os.path.join(modelFolderPath,"Illinois_ACC."+illinoisList[lastIndex]+"*.shp"))[0])
nbArray = mc.NaturalBreaks(df['normal_val'], k=6).bins.astype(float)

def classifyIllinois(value):   
    if value >= 0 and value <= nbArray[0]:
        return 0
    if value >= nbArray[0] and value <= nbArray[1]:
        return 1
    elif value > nbArray[1] and value <= nbArray[2]:
        return 2
    elif value > nbArray[2] and value <= nbArray[3]:
        return 3
    elif value > nbArray[3] and value <= nbArray[4]:
        return 4
    else:
        return 5
    
def classRangeIllinois(value):
    if value == 0:
        return "0~"+str(round(nbArray[0],2))
    if value == 1:
        return str(round(nbArray[0],2))+"~"+str(round(nbArray[1],2))
    elif value == 2:
        return str(round(nbArray[1],2))+"~"+str(round(nbArray[2],2))
    elif value == 3:
        return str(round(nbArray[2],2))+"~"+str(round(nbArray[3],2))
    elif value == 4:
        return str(round(nbArray[3],2))+"~"+str(round(nbArray[4],2))
    else:
        return str(round(nbArray[4],2))+"~"+str(round(nbArray[5],2))

illinoisACC_dissolve_list = []    

for i in range(0, len(illinoisList)):
    #In case we have multiple files for one day
    if (i != 0):
        if (illinoisList[i] == illinoisList[i-1]):
            continue
    illinoisACC = gpd.read_file(glob.glob(os.path.join(modelFolderPath,"Illinois_ACC."+illinoisList[i]+"*.shp"))[0])
    #illinoisACC = illinoisACC.rename(columns={"normal_cov": "normal_val"})
    
    illinoisACC['category'] = illinoisACC.apply(lambda x: classifyIllinois(x.normal_val), axis = 1)
    illinoisACC_dissolve = illinoisACC.dissolve(by='category', aggfunc = 'mean')
    illinoisACC_dissolve = illinoisACC_dissolve.drop(columns=['id'])
    illinoisACC_dissolve["category"] = range(0, len(illinoisACC_dissolve))
    illinoisACC_dissolve['date'] = datetime.strptime(illinoisList[i], "%m-%d-%Y").date().strftime("%Y-%m-%d")
    illinoisACC_dissolve['range'] = illinoisACC_dissolve.apply(lambda x: classRangeIllinois(x.category), axis = 1)
    #illinoisACC_dissolve["category"] = range(0, len(illinoisACC_dissolve))
    
    illinoisACC_dissolve = illinoisACC_dissolve.to_crs({'init': 'epsg:4326'})
    #illinoisACC_dissolve.to_file("Illinois_ACC."+illinoisList[i]+"_dissolve.geojson", driver="GeoJSON")
    
    illinoisACC_dissolve_list.append(illinoisACC_dissolve)
    
illinoisACC_concat = pd.concat(illinoisACC_dissolve_list, axis=0)
illinoisACC_concat = illinoisACC_concat.drop(columns=['category'])
illinoisACC_concat = illinoisACC_concat.reset_index()

#delta = timedelta(days=len(illinoisList))
firstDate = datetime.strptime(illinoisList[firstIndex], "%m-%d-%Y").date()
lastDate = datetime.strptime(illinoisList[lastIndex], "%m-%d-%Y").date()
illinoisACC_concat["dt_start"] = firstDate.strftime("%Y-%m-%d") #(lastDate-delta).strftime("%Y-%m-%d")
illinoisACC_concat["dt_end"] = lastDate.strftime("%Y-%m-%d")
illinoisACC_concat["dt_unit"] = "day"
illinoisACC_concat["cases_ts"] = "0,0,0,0,0,0,0"#str(np.zeros(len(illinoisList),int)).replace(" ",",")
illinoisACC_concat.to_file("Illinois_ACC_dissolve.geojson", driver="GeoJSON")

print("done")

done


In [75]:
illinoisACC_concat

,category,geometry,normal_val,date,range,dt_start,dt_end,dt_unit,cases_ts
0,0,"MULTIPOLYGON (((-88.43522 37.07011, -88.45102 ...",0.004031,2020-05-26,0~0.03,2020-05-26,2020-06-03,day,"0,0,0,0,0,0,0"
1,1,"MULTIPOLYGON (((-87.98429 38.58436, -87.96742 ...",0.046054,2020-05-26,0.03~0.07,2020-05-26,2020-06-03,day,"0,0,0,0,0,0,0"
2,2,"MULTIPOLYGON (((-90.27816 39.10560, -90.29378 ...",0.099779,2020-05-26,0.07~0.14,2020-05-26,2020-06-03,day,"0,0,0,0,0,0,0"
3,3,"MULTIPOLYGON (((-89.18925 38.97349, -89.17329 ...",0.308130,2020-05-26,0.14~0.43,2020-05-26,2020-06-03,day,"0,0,0,0,0,0,0"
4,4,"MULTIPOLYGON (((-90.39806 39.16972, -90.38246 ...",0.723234,2020-05-26,0.43~0.77,2020-05-26,2020-06-03,day,"0,0,0,0,0,0,0"
5,5,"MULTIPOLYGON (((-88.69178 39.07187, -88.67563 ...",0.923313,2020-05-26,0.77~1.0,2020-05-26,2020-06-03,day,"0,0,0,0,0,0,0"
6,0,"MULTIPOLYGON (((-88.43522 37.07011, -88.45102 ...",0.003994,2020-05-27,0~0.03,2020-05-26,2020-06-03,day,"0,0,0,0,0,0,0"
7,1,"MULTIPOLYGON (((-87.98429 38.58436, -87.96742 ...",0.045509,2020-05-27,0.03~0.07,2020-05-26,2020-06-03,day,"0,0,0,0,0,0,0"
8,2,"MULTIPOLYGON (((-90.27816 39.10560, -90.29378 ...",0.099830,2020-05-27,0.07~0.14,2020-05-26,2020-06-03,day,"0,0,0,0,0,0,0"
9,3,"MULTIPOLYGON (((-89.18925 38.97349, -89.17329 ...",0.308130,2020-05-27,0.14~0.43,2020-05-26,2020-06-03,day,"0,0,0,0,0,0,0"


In [76]:
# fig, ax = plt.subplots(figsize = (12,8))
# chicagoACC_dissolve.plot(column = 'category', edgecolor = "black", cmap='YlOrRd', ax=ax, legend = True)